# Query WordPress API for Themes by Last Update Date
### Test API Functionality by Fetching and Sorting Theme Counts by Update Date
WordPress API documentation: https://codex.wordpress.org/WordPress.org_API

In [1]:
import requests

themes_info = {}

# Define query API URL and query parameters
url = "https://api.wordpress.org/themes/info/1.2/"
params = {
    "action": "query_themes",
    "browse": "popular",  
    "per_page": 100,  
    "fields": [
        "last_updated"
    ],
}

# Start page count
page = 1

while True:
    # Add page number to params
    params["page"] = page
    
    # Request
    response = requests.get(url, params=params)
    
    # Verify the request was successful
    if response.status_code == 200:
        data = response.json()
        
        # Check if the response contains themes
        if "themes" in data:
            # Iteration among themes and add to the dictionary
            for theme in data["themes"]:
                slug = theme.get("slug")
                last_updated = theme.get("last_updated")
                
                # Store the slug and the last update date
                themes_info[slug] = {
                    "last_updated": last_updated
                }
            
            # If the amount of themes is lower than the allowed amount per page, then we are done
            if len(data["themes"]) < params["per_page"]:
                break
        else:
            break
    else:
        print(f"Error {response.status_code} when obtaining data.")
        break
    
    # Increase the page number by 1
    page += 1

# Print resulting dictionary
print(themes_info)

{'twentytwentyfive': {'last_updated': '2024-11-13'}, 'twentytwentyfour': {'last_updated': '2024-11-13'}, 'hello-elementor': {'last_updated': '2024-12-16'}, 'astra': {'last_updated': '2024-12-16'}, 'twentytwentythree': {'last_updated': '2024-11-13'}, 'twentytwentytwo': {'last_updated': '2024-11-13'}, 'twentytwentyone': {'last_updated': '2024-11-13'}, 'kadence': {'last_updated': '2024-12-17'}, 'oceanwp': {'last_updated': '2024-12-18'}, 'twentytwenty': {'last_updated': '2024-11-13'}, 'twentyseventeen': {'last_updated': '2024-11-12'}, 'generatepress': {'last_updated': '2024-09-04'}, 'neve': {'last_updated': '2024-12-06'}, 'blocksy': {'last_updated': '2024-12-12'}, 'kubio': {'last_updated': '2024-11-21'}, 'extendable': {'last_updated': '2024-12-19'}, 'futurio-storefront': {'last_updated': '2024-10-27'}, 'envo-one': {'last_updated': '2024-11-12'}, 'royal-elementor-kit': {'last_updated': '2024-11-14'}, 'yith-wonder': {'last_updated': '2024-08-06'}, 'popularfx': {'last_updated': '2024-11-19'},

## Aggregate data using their last_updated info

In [2]:
from datetime import datetime, timedelta

# Calculate amount of times depending on the criteria
def categorize_themes(themes_info):
    # Current date
    now = datetime.now()
    
    # Crear los contenedores para cada categoría
    all_themes = 0
    updated_last_week = 0
    updated_last_month = 0
    updated_last_year = 0
    
    # Turn dates to datetime format
    for slug, info in themes_info.items():
        last_updated_str = info.get("last_updated")
        
        if last_updated_str:
            last_updated = datetime.strptime(last_updated_str, "%Y-%m-%d")
            all_themes += 1
            
            # Compare current date with given old dates
            if now - last_updated <= timedelta(weeks=1):
                updated_last_week += 1
            if now - last_updated <= timedelta(days=30):
                updated_last_month += 1
            if now - last_updated <= timedelta(days=365):
                updated_last_year += 1

    # Return resulting data in a dictionary
    return {
        "total_themes": all_themes,
        "updated_last_week": updated_last_week,
        "updated_last_month": updated_last_month,
        "updated_last_year": updated_last_year,
    }

# Get theme count according to criteria
theme_counts = categorize_themes(themes_info)

# Print the result
print(theme_counts)


{'total_themes': 7139, 'updated_last_week': 751, 'updated_last_month': 1834, 'updated_last_year': 5779}


## Retrieve the Theme with the Oldest Update Date

In [3]:
# Declare variables
oldest_theme = None
oldest_update = None

# Check all themes and gran the oldest date
for slug, info in themes_info.items():
    last_updated_str = info.get("last_updated")
    
    if last_updated_str:
        last_updated = datetime.strptime(last_updated_str, "%Y-%m-%d")
        
        # Si es el primer tema o tiene una fecha más antigua, actualizar la variable
        if oldest_update is None or last_updated < oldest_update:
            oldest_update = last_updated
            oldest_theme = slug

# Print theme with oldest date
print(f"The theme with the oldest update is '{oldest_theme}', and the date is: {oldest_update}")


The theme with the oldest update is 'hyakutake', and the date is: 2022-12-20 00:00:00
